In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation as ani
import numpy as np
import plotly.express as px

In [3]:
#https://www.youtube.com/watch?v=KAmZe5D3v5I

#RDG tuples for colours of health status
GREY = (0, 0, 0, 0.5) #uninfected
RED = (255, 0, 0, 0.5) #infected
GREEN = (0, 255, 0, 0.5) #recovered
BLACK = (0, 0, 0, 1) #dead- may not need this one as just part of 'recovered' for hospitable admissions

#creating a dictionary with info specific to covid- find an internet source for thse values-not correct yet!!
parameters = {
    'r0': #can we use basic SIR covid simulation here to work this out??,
    'incubation_period': ,
    'recovery_out_hospital': (x,y) ,# x, y, z, k all values to research for min to max no days spent revoering in the two environments
    'recovery_in_hospital': (z,k),
    'percent_infected_out_hospital': ,
    'percent_infected_in_hospital': ,
}

class Covid_model_fun():
    def __init__(self, parameters):
        #creating line graph of daily hospital admissions of our simulation AND real time data
        self.fig = px.line(df, x='day', y='number of hospital admissions', title='hospital admissions per day')
        self.real_data = self.fig.add_subplot(#add the real_data line here)
        self.fig.show()
        
        #shiiiiiiiiiitttttttt
    
        self.day = 0
        self.total_num_infected = 0
        self.num_currently_infected = 0
        self.num_recovered = 0  #includes deaths in this
        self.r0 = params['r0']
        self.percent_infected_out_hospital = params['percent_infected_out_hospital']
        self.percent_infected_in_hospital = params['percent_infected_in_hospital']
        
        self.in_hospital_slow = params['incubation_period'] + params[recovery_in_hospital][1]
        self.in_hospital_fast = params['incubation_period'] + params[recovery_in_hospital][0]
        self.out_hospital_slow = params['incubation_period'] + params[recovery_out_hospital][1]
        self.out_hospital_fast = params['incubation_period'] + params[recovery_out_hospital][0]
        
        #dictionary for recovery time for infected people in and out of hospital over a year
        self.out_hospital = {i: {} for i in range(self.out_hospital_fast, 365)}
        self.in_hospital = {i: {} for i in range(self.in_hospital_fast, 365)}
        
        #restricting spread of virus to a subset of population
        self.exposed_start = 0 #no people exposed already
        self.exposed_end = 1 #tot no of people exposed by the end of the wave
    
    def initial_population(self):
        population = 56500000  #of England at beginning of 2020
        self.total_num_infected = 1
        self.num_currently_infected = 1
        #indices = np.arrange(0, population) + 0.5

Covid_model_fun(parameters)
plt.show()

SyntaxError: invalid syntax (<ipython-input-3-79e031df1124>, line 12)